In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport
from pandas.api.types import CategoricalDtype

%matplotlib inline

In [2]:
survey_data_file = "../data/survey_data.csv"
app_usage_file = "../data/survey_users_app_usage.csv"

In [4]:
df_survey_raw = pd.read_csv(survey_data_file)
# df_app = pd.read_csv(app_usage_file, parse_dates=['duolingo_start_date'])

In [10]:
df_survey_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6187 entries, 0 to 6186
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   user_id                               6187 non-null   object
 1   age                                   5838 non-null   object
 2   annual_income                         5182 non-null   object
 3   country                               6187 non-null   object
 4   duolingo_platform                     5911 non-null   object
 5   duolingo_subscriber                   5901 non-null   object
 6   duolingo_usage                        5911 non-null   object
 7   employment_status                     5730 non-null   object
 8   future_contact                        5446 non-null   object
 9   gender                                5838 non-null   object
 10  other_resources                       4474 non-null   object
 11  primary_language_commitment   

In [6]:
unique_counts = pd.DataFrame.from_records([(col, df_survey_raw[col].nunique()) for col in df_survey_raw.columns],
                          columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])

In [7]:
unique_counts

,Column_Name,Num_Unique
8,future_contact,2
12,primary_language_review,2
17,survey_complete,2
15,primary_language_proficiency,3
16,student,3
4,duolingo_platform,3
5,duolingo_subscriber,4
7,employment_status,4
9,gender,4
6,duolingo_usage,5


In [108]:
df_survey = df_survey_raw.copy()

In [109]:
#only keep results where survey was completed
total_rows = df_survey.shape[0]
completed_surveys = df_survey['survey_complete'].sum()
print('The data contains {} entries, but only {} where survey_complete is True. Dropping {} rows' \
      .format(total_rows, completed_surveys, total_rows-completed_surveys))

The data contains 6187 entries, but only 5729 where survey_complete is True. Dropping 458 rows


In [110]:
index_names = df_survey[df_survey['survey_complete']==0].index
df_survey.drop(index_names, inplace=True)

In [111]:
# I don't care how long it took people to complete the survey, so dropping that column
# Also, we've already filterd on survey_complete == 1, so dropping that column
# Also don't care if we can contact the user again
df_survey.drop(columns=['time_spent_seconds', 'survey_complete', 'future_contact'], inplace=True)
# 'primary_language_motivation_followup' has high cardinality and lots of NaN's - not useful here
df_survey.drop(columns=['primary_language_motivation_followup'], inplace=True)

In [112]:
# Now let's find duplicate entries by user_id
duplicate_users = df_survey[df_survey.duplicated(['user_id'])]
# duplicate_users.shape[0] -> there are 35 duplicates
# Let's drop the duplicates, and choose to keep the first entry per user
df_survey.drop_duplicates(subset=['user_id'], keep='first', inplace=True)

In [123]:
column_to_check = 'primary_language_commitment'
print('Unique values in column are: ', [x for x in df_survey[column_to_check].unique()])
print('Total NaNs = {}'.format(df_survey[column_to_check].isnull().sum()))

Unique values in column are:  ["I'm very committed to learning this language.", "I'm slightly committed to learning this language.", "I'm moderately committed to learning this language.", "I'm extremely committed to learning this language.", "I'm not at all committed to learning this language.", nan]
Total NaNs = 6


In [114]:
# Now let's see if we can fill some missing values
df_survey['duolingo_platform'].fillna('Unknown', inplace=True)
df_survey['duolingo_subscriber'].fillna("I don't know if I pay for Duolingo Plus", inplace=True)
df_survey['duolingo_usage'].fillna("I don't use Duolingo", inplace=True)
# with 110 nan's, leaving 'employment_status' NaN for now - to be imputed later
# 'age' has 9 Nan's, but is highly correlated with 'employment_status' and 'student' so will try to impute
# 'annual_income' has 633 NaN's - will try to impute later
df_survey['gender'].fillna("Prefer not to say", inplace=True) # they didn't answer the question, and only 10 NaN's
# 'primary_language_commitment' has only 6 NaN's - leaving those for now
# 'primary_language_review' has only 13 NaN's - leaving those for now
# 'primary_language_motivation' has only 15 NaN's - leaving those for now
# 'primary_language_proficiency' has only 4 NaN's - leaving those for now
# 'student' has 307 NaN's - will try to impute

In [115]:
# Next, let's encode some categorical variables where ordering doesn't matter
df_survey['country'] = df_survey['country'].astype('category')
df_survey['duolingo_platform'] = df_survey['duolingo_platform'].astype('category')
df_survey['duolingo_subscriber'] = df_survey['duolingo_subscriber'].astype('category')
df_survey['employment_status'] = df_survey['employment_status'].astype('category')
df_survey['gender'] = df_survey['gender'].astype('category')
df_survey['student'] = df_survey['student'].astype('category')

In [118]:
# Next, mapping ordinal categories where ordering does matter
# 'duolingo_usage'
usage_ordering = ["I don't use Duolingo", "Less than once a month", "Monthly", "Weekly", "Daily"]
usage_type = CategoricalDtype(categories=usage_ordering, ordered=True)
df_survey['duolingo_usage'] = df_survey['duolingo_usage'].astype('category')
df_survey['duolingo_usage'] = df_survey['duolingo_usage'].cat.reorder_categories(usage_ordering, ordered=True)

# 'age'
age_ordering = ['Under 18', '18-34', '35 - 54', '55 - 74', '75 or older']
age_type = CategoricalDtype(categories=age_ordering, ordered=True)
df_survey['age'] = df_survey['age'].astype('category')
df_survey['age'] = df_survey['age'].cat.reorder_categories(age_ordering, ordered=True)

# 'annual_income'
income_ordering = ['$0 - $10,000', '$11,000 - $25,000', '$26,000 - $75,000', '$76,000 - $150,000', '$151,000 or more']
income_type = CategoricalDtype(categories=income_ordering, ordered=True)
df_survey['annual_income'] = df_survey['annual_income'].astype('category')
df_survey['annual_income'] = df_survey['annual_income'].cat.reorder_categories(income_ordering, ordered=True)

# 'primary_language_commitment'
commitment_ordering = ["I'm not at all committed to learning this language.",
                       "I'm slightly committed to learning this language.", 
                       "I'm moderately committed to learning this language.", 
                       "I'm very committed to learning this language.", 
                       "I'm extremely committed to learning this language."]
commitment_type = CategoricalDtype(categories=commitment_ordering, ordered=True)
df_survey['primary_language_commitment'] = df_survey['primary_language_commitment'].astype('category')
df_survey['primary_language_commitment'] = df_survey['primary_language_commitment'].cat.reorder_categories(commitment_ordering, ordered=True)

# 'primary_language_commitment'
commitment_ordering = ["I'm not at all committed to learning this language.",
                       "I'm slightly committed to learning this language.", 
                       "I'm moderately committed to learning this language.", 
                       "I'm very committed to learning this language.", 
                       "I'm extremely committed to learning this language."]
commitment_type = CategoricalDtype(categories=commitment_ordering, ordered=True)
df_survey['primary_language_commitment'] = df_survey['primary_language_commitment'].astype('category')
df_survey['primary_language_commitment'] = df_survey['primary_language_commitment'].cat.reorder_categories(commitment_ordering, ordered=True)

['Daily', 'Weekly', 'Monthly', 'Less than once a month', 'I don't use Duolingo']
Categories (5, object): ['I don't use Duolingo' < 'Less than once a month' < 'Monthly' < 'Weekly' < 'Daily']